In [2]:
from elasticsearch import Elasticsearch

import shared.config as config
import shared.const as const
import pymongo
import numpy as np
import pandas as pd
import json
import re
import ast
import logging

from pymongo import ReplaceOne
import sqlalchemy as db
from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine, inspect
from sqlalchemy.engine import reflection
#from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Text, ForeignKey, Float
from sqlalchemy.orm import relationship, Session, declarative_base
from sqlalchemy.sql import column, text
from sqlalchemy import select, func
from sqlalchemy.sql.expression import literal

logger = logging.getLogger()

In [19]:

Base = declarative_base()
index_name = "abr_test2"

class WasstraatModel(Base):
    __abstract__ = True

    herkomst = Column(Text)
    key = Column(Text)
    soort = Column(String(80))
    brondata = Column(Text)
    uuid = Column('_id', String)

class Project(WasstraatModel): # Inherit from Model for cannot use Abstract class Wasstraatmodel, for geo-package gives errors
    __tablename__ = 'Def_Project'

    primary_key = Column(Integer, primary_key=True, autoincrement=True)
    projectcd = Column(String(12), nullable=False, index=True)
    projectnaam = Column(String(1024), index=True)
    jaar = Column(Integer)
    toponiem = Column(String(1024))
    trefwoorden = Column(String(1024))
    #location = Column(Geometry('POINT', srid=4326), default=(52.00667, 4.35556)) # 52.00667, 4.35556.
    xcoor_rd = Column(Float)
    ycoor_rd = Column(Float)
    longitude = Column(Float)
    latitude = Column(Float)
    #artefacten = relationship("Artefact", back_populates="project")

    # Explicit defined for cannot use Abstract class Wasstraatmodel, for geo-package gives errors
    herkomst = Column(Text)
    soort = Column(String(80))
    brondata = Column(Text)
    uuid = Column('_id', String)
    key = Column(Text)

    @staticmethod
    def getDescription(p):
        return str(p.projectcd) + f' ({p.projectnaam if p.projectnaam and p.projectnaam != "" else "Zonder naam"})'

    def __repr__(self):
        return Project.getDescription(self)


engine = create_engine(config.SQLALCHEMY_DATABASE_URI)    
session = Session(engine)

#query = session.query(User).filter(User.name.like("%ed")).order_by(User.id)

q = session.query(Project).filter(Project.projectcd.like('DC%'))
result = q.all()

df = pd.DataFrame(result)
df

,0
0,DC001 (Oude Delft 96)
1,DC002 (Oude Delft 188)
2,DC003 (Markt 1-3)
3,DC004 (Waag)
4,DC005 (Anna Convent OLD74)
...,...
251,DC254 (IVO-P Bastiaanpoort)
252,DC255 (DO Bastiaanpoort)
253,DC256 (Waarneming Van der Mastenstraat)
254,DC257 (Barbaraklooster 1960)


In [5]:
from sqlalchemy.dialects import postgresql

pkey_json = [{'primary_key': hit['_id'], 'score': hit['_score']} for hit in resp['hits']['hits']]
pkey_json = json.dumps(pkey_json) 
print(pkey_json)

engine = create_engine(config.SQLALCHEMY_DATABASE_URI)    
session = Session(engine)
result = session.execute(sql)

q = session.query(Project).filter(Project.projectcd.like('DC%'))
subquery = text(f"SELECT * FROM json_to_recordset('{pkey_json}')" + ' as x("primary_key" int, "score" float)')
subquery= subquery.columns(column('primary_key'), column('score'))
subquery = subquery.subquery()
q = q.join(subquery, Project.primary_key == subquery.c.primary_key).add_columns(subquery.c.score.label("fulltext_score")).order_by(subquery.c.score.desc())
print(str(q.statement.compile(dialect=postgresql.dialect())))

df = pd.DataFrame(q.all())
df

[]


NameError: name 'sql' is not defined

In [50]:
es = Elasticsearch('http://elasticsearch:9200')
resp = es.search(index='def_project', size=500, query={'simple_query_string': {'query': '*dc22*', 'fields': ['*projectcd'], 'analyze_wildcard':True}})
print("Got %d Hits:" % resp['hits']['total']['value'])
for hit in resp['hits']['hits']:
    print(hit)


Got 10 Hits:
{'_index': 'def_project', '_id': '431', '_score': 1.0, '_source': {'doc': {'primary_key': '431', 'projectcd': 'DC220', 'projectnaam': 'Muziekschool', 'toponiem': 'Muziekschool Gasthuislaan', 'trefwoorden': '', 'soort': 'Project', 'id': 431}}}
{'_index': 'def_project', '_id': '432', '_score': 1.0, '_source': {'doc': {'primary_key': '432', 'projectcd': 'DC221', 'projectnaam': 'Opgraving Oude Delft 54', 'toponiem': 'Oude Delft 54', 'trefwoorden': '', 'soort': 'Project', 'id': 432}}}
{'_index': 'def_project', '_id': '433', '_score': 1.0, '_source': {'doc': {'primary_key': '433', 'projectcd': 'DC222', 'projectnaam': 'Oude kerk en prinsenhof', 'toponiem': 'Profiel Oude Delft thv Prinsenhof', 'trefwoorden': '', 'soort': 'Project', 'id': 433}}}
{'_index': 'def_project', '_id': '434', '_score': 1.0, '_source': {'doc': {'primary_key': '434', 'projectcd': 'DC223', 'projectnaam': 'BO Nieuw Delft veld 5', 'toponiem': 'Nieuw Delft veld 5', 'trefwoorden': '', 'soort': 'Project', 'id': 43

In [6]:
resp = es.search(index='my_index', query={'simple_query_string': {'query': 'scond~'}})
print("Got %d Hits:" % resp['hits']['total']['value'])
for hit in resp['hits']['hits']:
    print(hit)

Got 1 Hits:
{'_index': 'my_index', '_id': '2', '_score': 0.5304083, '_source': {'text': 'this is a second test', 'test2': 'nog iets anders'}}


In [7]:
lst_skipindex = ['_id', 'herkomst', 'key', 'brondata']


def generate_docs(resultset, db_col_names, index_name):
    for row in resultset:
        doc = {
            "_index": index_name,
            "_id": int(row["primary_key"]),
            "doc": {col.lower(): str(row[col] if str(row[col]) != 'None' else '') for col in db_col_names}
        }
        doc['doc']['id'] = int(row["primary_key"])
        yield doc


def indexTable(table):
    logger.info(f"Starting indexing table {table} for fulltext indexing...")

    engine = create_engine(config.SQLALCHEMY_DATABASE_URI)
    logger.info("Connecting to " + config.SQLALCHEMY_DATABASE_URI)
    es = Elasticsearch('http://elasticsearch:9200')
    
    index_name = table.lower()

    with engine.connect() as connection:
        connection = connection.execution_options( 
            isolation_level="SERIALIZABLE",
            postgresql_deferrable=True # Does not seem to work. Work imn progress 
        )
        with connection.begin():
            metadata = db.MetaData(bind=engine)
            db.MetaData.reflect(metadata)

            #get the table list
            dict_tables = metadata.tables.keys()
            lst_tables = [x for x in list(dict_tables) if x.startswith('Def_')]
            logger.info("Indexing all string fields for " + str(lst_tables))

            if table in lst_tables:
                insp = inspect(connection)
                db_cols = insp.get_columns(table)
                db_cols = [col for col in db_cols if (col['name'] == 'primary_key' or 'text' in str(col['type']).lower() or 'varchar' in str(col['type']).lower()) and col['name'] not in lst_skipindex]
                sql_col_names = ['"'+col['name']+ '"' for col in db_cols]
                db_col_names = [col['name'] for col in db_cols]
                
                sql = f'SELECT {", ".join(sql_col_names)} FROM public."{table}"'
                logger.info(f"Indexing index {index_name} with columns {db_col_names} ")
                rs = connection.execute(sql)
                
                logger.info(f"Indexing index {index_name} with columns {db_col_names} ")
                helpers.bulk(es, generate_docs(rs, db_col_names, index_name))
                logger.info(f"Indexed {es.count(index=index_name)} records")                
            else:
                logger.error(f"Trying to index table {table}, but table not available in {lst_tables}")

es = Elasticsearch('http://elasticsearch:9200')
indexTable('Def_Artefact')

TypeError: MetaData.__init__() got an unexpected keyword argument 'bind'

In [8]:
es.options(ignore_status=[400,404]).indices.delete(index='_all')

ObjectApiResponse({'error': {'root_cause': [{'type': 'illegal_argument_exception', 'reason': 'Wildcard expressions or all indices are not allowed'}], 'type': 'illegal_argument_exception', 'reason': 'Wildcard expressions or all indices are not allowed'}, 'status': 400})

In [9]:
from sqlalchemy import select, func
stmt = select(func.json_each('{"a":"foo", "b":"bar"}').table_valued("key", "value"))
print(stmt)

session.rollback()
result = session.execute(stmt)
df = pd.DataFrame(result)
df


SELECT anon_1.key, anon_1.value 
FROM json_each(:json_each_1) AS anon_1


,key,value
0,a,foo
1,b,bar


In [10]:
from sqlalchemy import select, func, column, Integer, Text
stmt = select(
    func.json_to_recordset(str(pkey_json)).table_valued(
        column("primary_key", Integer), column("score", Float),
    ).render_derived(name="t", with_types=True)
)
print(stmt)

#session.rollback()
#result = session.execute(stmt)
#df = pd.DataFrame(result)
#df
subquery = session.query(func.json_to_recordset(str(pkey_json)).\
                             table_valued(column("primary_key", Integer), column("score", Float)).\
                             render_derived(name="t", with_types=True)).\
                             subquery()

'''
sub_query = session.query(
    0.9 * func.sum(Revenue.total_revenue)
).select_from(
    Revenue
).filter(
    func.datediff(func.now(), Revenue.date) <= 7
).subquery()
'''

q = session.query(Project).filter(Project.projectcd.like('DC%'))
#subquery = text(f"SELECT * FROM json_to_recordset('{pkey_json}')" + ' as x("primary_key" int, "score" float)')
#subquery= subquery.columns(column('primary_key'), column('score'))
#subquery = subquery.subquery()
q = q.join(subquery, Project.primary_key == subquery.c.primary_key).add_columns(subquery.c.score.label("fulltext_score")).order_by(subquery.c.score.desc())
print(str(q.statement.compile(dialect=postgresql.dialect())))

df = pd.DataFrame(q.all())
df

SELECT t.primary_key, t.score 
FROM json_to_recordset(:json_to_recordset_1) AS t(primary_key INTEGER, score FLOAT)
SELECT "Def_Project".primary_key, "Def_Project".projectcd, "Def_Project".projectnaam, "Def_Project".jaar, "Def_Project".toponiem, "Def_Project".trefwoorden, "Def_Project".xcoor_rd, "Def_Project".ycoor_rd, "Def_Project".longitude, "Def_Project".latitude, "Def_Project".herkomst, "Def_Project".soort, "Def_Project".brondata, "Def_Project"._id, "Def_Project".key, anon_1.score AS fulltext_score 
FROM "Def_Project" JOIN (SELECT t.primary_key AS primary_key, t.score AS score 
FROM json_to_recordset(%(json_to_recordset_1)s) AS t(primary_key INTEGER, score FLOAT)) AS anon_1 ON "Def_Project".primary_key = anon_1.primary_key 
WHERE "Def_Project".projectcd LIKE %(projectcd_1)s ORDER BY anon_1.score DESC


""


In [52]:
conversion_table = (
    (
        "is_relation_many_to_one",
        ['FilterRelationOneToManyEqual', 'FilterRelationOneToManyNotEqual'],
    ),
    (
        "is_relation_one_to_one",
        ['FilterRelationOneToManyEqual', 'FilterRelationOneToManyNotEqual'],
    ),
    (
        "is_text",
        [
            'FilterStartsWith',
            'FilterEndsWith',
        ],
    )
)
conversion_table = dict(conversion_table)
#conversion_table['is_text'] = conversion_table['is_text'] + ['Test']


#conversion_table = tuple([(k, v) for k, v in conversion_table.items()])
conversion_table

{'is_relation_many_to_one': ['FilterRelationOneToManyEqual',
  'FilterRelationOneToManyNotEqual'],
 'is_relation_one_to_one': ['FilterRelationOneToManyEqual',
  'FilterRelationOneToManyNotEqual'],
 'is_text': ['FilterStartsWith', 'FilterEndsWith']}

In [54]:
'is_relation_many_to_one' in conversion_table.keys()

True

In [1]:
None or 'appel'

'appel'